# Lab6: Convex Optimization 

Outline:
1. Convex optimization problem.
2. Why convex optimization.
3. Examples

-----

# Using `cvxpy` for numerical optimization

## 1. Convex optimization problem
\begin{align*}
    \mbox{minimize} & \ f_{0}(x) \\
    \mbox{subject to} & \ f_{i}(x)\leq 0, i=1,\ldots,m \\
                      & Ax = b
\end{align*}

with variable $x\in \mathbb{R}^{n}$
- Objective and inequality constraints $f_{0},\ldots,f_{m}$ are convex; i.e., graphs of $f_{i}$ curve upward.
- Equality constraints are *linear*.

## 2. Why convex optimization?

- Beautiful, fairly complete, and useful theory.
- Solution algorithms that work well in theory and practice.
- many applications in:
    + [machine learning](http://nbviewer.jupyter.org/github/cvxgrp/cvx_short_course/blob/master/intro/SVM.ipynb), [statistics](http://nbviewer.jupyter.org/github/cvxgrp/cvx_short_course/blob/master/applications/quantile_regression.ipynb)
    + [control](http://nbviewer.jupyter.org/github/cvxgrp/cvx_short_course/blob/master/intro/control.ipynb)
    + [signal processing](http://nbviewer.jupyter.org/github/cvxgrp/cvxpy/blob/master/examples/notebooks/WWW/robust_kalman.ipynb)
    + [finance](http://nbviewer.jupyter.org/github/cvxgrp/cvx_short_course/blob/master/applications/portfolio_optimization.ipynb)
    
Very complete reference of convex optimization: https://web.stanford.edu/~boyd/cvxbook/

# 3. Examples

## 3.1 Basic Example: Maximum Likelihood Estimation

Suppose we have $n$ IDD realizations, $X_{1},\ldots,X_{n}$, of a Poisson random variable with rate $\lambda>0$ with pmf:
$$
p(x_{i}):= e^{-\lambda} \frac{\lambda^{x_{i}}}{x_{i}!}, \ \lambda>0, \ x_{i}=0,1,2,\ldots
$$

The log-likelihood function in $\lambda$ is thus given by:
\begin{align*}
l(\lambda) &:= \log \prod_{i=1}^{n} p(x_{i}) \\
&= -n\lambda + \log\lambda \sum_{i=1}^{n}x_{i} - \sum_{i=1}^{n}\log(x_{i}!)
\end{align*}

In order to find the MLE of $\lambda$ we need to solve the following contraint optimization problem:

$$
\begin{array}{ll}
\mbox{maximize}   & l(\lambda) \\
\mbox{subject to} & \lambda >0 
\end{array}
$$

In [ ]:
# Generate data
import numpy as np
import scipy as sp
n = 100
lambda_true = 2
x = np.random.poisson(lam=lambda_true,size=n)

In [ ]:
import cvxpy as cvx

- We start by declaring out the optimization [variable](http://www.cvxpy.org/en/latest/tutorial/intro/index.html#vectors-and-matrices):

In [ ]:
# Create optimization variable.
lambda_ = cvx.Variable() # lambda is a python-specific variable so we use lambda_

- We then define the the set of [contraints](http://www.cvxpy.org/en/latest/tutorial/intro/index.html#constraints) that apply to our optimization problem:

In [ ]:
constraints = [lambda_>=0] # Strict inequalities are not allowed.

- We construct the objective function (i.e., function to be optimized) using the admisible set of `cvx` [functions](http://www.cvxpy.org/en/latest/tutorial/functions/index.html):

In [ ]:
obj = cvx.Maximize(-n*lambda_ + \
                   np.sum(x)*cvx.log(lambda_) - \
                   np.sum(np.log(sp.special.factorial(x))))

- Form and solve the optimization problem:

In [ ]:
prob = cvx.Problem(obj, constraints)
prob.solve()  # Returns the optimal VALUE (i.e., the value of the objective at the maximum).

We now obtain the actual maximizer of the log-likelihood:

In [ ]:
lambda_.value # which is close to the true value  lambda=2

## 3.2 Nonnegative matrix factorization as a convex optimization problem

A derivative work by Judson Wilson, 6/2/2014.    
Adapted from the CVX example of the same name, by Argyris Zymnis, Joelle Skaf and Stephen Boyd

Source: https://github.com/cvxgrp/cvxpy/blob/master/examples/notebooks/WWW/nonneg_matrix_fact.ipynb

### Introduction

We are given a matrix $X \in \mathbf{\mbox{R}}^{v \times n}$ and are interested in solving the problem:
    \begin{array}{ll}
    \mbox{minimize}   & \| X - WH \|_F \\
    \mbox{subject to} & W \succeq 0 \\
                      & H \succeq 0,
    \end{array}
where $W \in \mathbf{\mbox{R}}^{v \times r}$ and $H \in \mathbf{\mbox{R}}^{r \times n}$.

This example generates a random matrix $X$ and obtains an
*approximate* solution to the above problem by first generating
a random initial guess for $W$ and then alternatively minimizing
over $W$ and $H$ for a fixed number of iterations.

### Generate problem data

In [ ]:
import cvxpy as cvx
import numpy as np

# Ensure repeatably random problem data.
np.random.seed(0)

# Generate random data matrix X.
v = 10 # e.g. number of frequency cells in our NBA example
n = 10 # e.g. number of players
r = 5  # e.g. number of bases vectors
X = np.random.rand(v, r).dot(np.random.rand(r, n))

# Initialize Y randomly.
W_init = np.random.rand(v, r)

### Perform alternating minimization

In [ ]:
# Ensure same initial random Y, rather than generate new one
# when executing this cell.
W = W_init 

# Perform alternating minimization.
MAX_ITERS = 30
residual = np.zeros(MAX_ITERS)

for iter_num in range(1, 1+MAX_ITERS):
    # At the beginning of an iteration, X and Y are NumPy
    # array types, NOT CVXPY variables.

    # For odd iterations, treat Y constant, optimize over X.
    if iter_num % 2 == 1:
        H = cvx.Variable(r, n)
        constraint = [H >= 0]
        
    # For even iterations, treat X constant, optimize over Y.
    else:
        W = cvx.Variable(v, r)
        constraint = [W >= 0]
    
    # Solve the problem.
    obj = cvx.Minimize(cvx.norm(X - W*H, 'fro'))
    prob = cvx.Problem(obj, constraint)
    prob.solve(solver=cvx.SCS)

    if prob.status != cvx.OPTIMAL:
        raise Exception("Solver did not converge!")
    
    print('Iteration {}, residual norm {}'.format(iter_num, prob.value))
    residual[iter_num-1] = prob.value

    # Convert variable to NumPy array constant for next iteration.
    if iter_num % 2 == 1:
        H = H.value
    else:
        W = W.value

## 3.3 Ridge regression

We wish to recover a sparse vector $\beta\in \mathbb{R}^{n}$ from measurements $y\in \mathbb{R}^{m}$. Our measurement model tells us that 
$$
y=X\beta+\epsilon,
$$
where $X\in\mathbb{R}^{m\times n}$ is a known design matrix and $\epsilon \in \mathbb{R}^{m}$ is unknown measurement error. 

The entries of $\epsilon$ are drawn IID from the distribution $\mathcal{N}(0,\sigma^{2})$.

We can first try to recover $x$ by solving the optimization problem
$$
\mbox{minimize} \ ||y-X\beta||^{2}_{2}+\gamma||\beta||^{2}_{2}
$$

This problem is called *ridge* regression.

The code below defines $n$, $m$, $X$, $\beta$, and $y$. 

We use CVXPY to estimate $\beta$ from $y$ using ridge regression for a given value of $\gamma$, where we moreover try multiple values of $\gamma$.


In [ ]:
import cvxpy  as cvx
import numpy
import matplotlib.pyplot as plt

# Problem data.
n = 15
m = 10
numpy.random.seed(1)
X = numpy.random.randn(n, m)
y = numpy.random.randn(n)

- We can define arbitrary [parameters](http://www.cvxpy.org/en/latest/tutorial/intro/index.html#parameters) to be part of our objective function:

In [ ]:
# gamma must be positive due to DCP rules.
gamma = cvx.Parameter(sign="positive") 

Parameters are symbolic representations of constants. The purpose of parameters is to change the value of a constant in a problem without reconstructing the entire problem.

In [ ]:
# Construct the problem.
beta_ = cvx.Variable(m) # independent variable
sum_of_squares = sum(cvx.square(X*beta_ - y))
obj = cvx.Minimize(sum_of_squares + gamma*cvx.norm(beta_,2))
prob = cvx.Problem(obj)

In [ ]:
gamma.value = 1/2

In [ ]:
prob.solve() # objective evaluated at solution

In [ ]:
beta_.value # estimated parameters for gamma = 1/2

In [ ]:
# Now repeat same procedure for several values of gamma,
# in practice you would use cross-valudation to obtain the optimal value for gamma:
gamma_vals = numpy.logspace(-4, 6) 
gamma_vals

The plot below may take a while to excecute but we can visualize the resulting image here:
http://www.cvxpy.org/en/latest/tutorial/intro/index.html#parameters


In [ ]:
# Construct a trade-off curve of ||Ax-b||^2 vs. ||x||_1
sq_penalty = []
l2_penalty = []
beta_values = []
gamma_vals = numpy.logspace(-4, 6)
for val in gamma_vals:
    gamma.value = val
    prob.solve()
    # Use expr.value to get the numerical value of
    # an expression in the problem.
    sq_penalty.append(sum_of_squares.value)
    l2_penalty.append(cvx.norm(beta_, 1).value)
    beta_values.append(beta_.value)

In [ ]:
plt.figure(figsize=(6,10))

# Plot trade-off curve.
plt.subplot(211)
plt.plot(l2_penalty, sq_penalty)
plt.xlabel(r'$\||\beta\||_2$', fontsize=16) # we can include latex notation in labels
plt.ylabel(r'$\||y- X\beta\||^{2}_{2}$', fontsize=16)
plt.title('Trade-Off Curve for Ridge Regression', fontsize=16)

# Plot entries of beta vs. gamma.
plt.subplot(212)
for i in range(m):
    plt.plot(gamma_vals, [xi[i,0] for xi in beta_values])
plt.xlabel(r'$\gamma$', fontsize=16)
plt.ylabel(r'$\beta_{i}$', fontsize=16)
plt.xscale('log')
plt.title(r'Entries of $\beta$ vs. $\gamma$', fontsize=16)

plt.tight_layout()
plt.show()